<a href="https://colab.research.google.com/github/hkjm/rdlenia/blob/main/RDLenia_cupy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import IPython.display
from IPython.display import display, clear_output
import scipy
from PIL import Image

!pip -q install "cupy-cuda12x"
import cupy as cp
from cupyx.scipy import signal

from pathlib import Path, PosixPath
import urllib.request, os

DATA_DIR = Path("data")
FNAME    = "initialPattern.txt"
URL      = f"https://raw.githubusercontent.com/hkjm/rdlenia/main/{DATA_DIR}/{FNAME}"
path = DATA_DIR / FNAME
if not path.exists():
    os.makedirs(DATA_DIR, exist_ok=True)
    urllib.request.urlretrieve(URL, path)

_c0 = np.loadtxt(path)

In [ ]:
# parameters

sizeX = sizeY = 128
numSp = 40
mu        = 0.1

tau_learn = 0.00001
tau_sim   = 0.00001
epsilon_learn   = 0.005
epsilon_sim   = 0.005

n_learn   = 10000
n_sim     = 2000000
save_every = 10000

m, s      = 0.21, 0.018
bell = lambda x, m, s: np.exp(-((x-m)/s)**2 / 2)

b = np.asarray([5/6,7/12,1])
R = 36
D = np.linalg.norm(np.array(np.meshgrid(np.arange(-R,R), np.arange(-R,R)))+1,axis=0)  / R * len(b)
K = (D<len(b)) * b[np.minimum(D.astype(int),len(b)-1)] * bell(D % 1, 0.5, 0.15)
K = K/np.sum(K)

bell_cp = lambda x, m, s: cp.exp(-((x-m)/s)**2 / 2)
def target_cp(U): return bell_cp(U, m, s)
K_cp = cp.asarray(K, dtype=cp.float64)

D_vec = cp.arange(1, numSp+1, dtype=cp.float64)[:,None,None]
def laplacian_all(v):
    v_u = cp.roll(v, -1, 1); v_d = cp.roll(v, 1, 1)
    v_l = cp.roll(v, -1, 2); v_r = cp.roll(v, 1, 2)
    return v_u + v_d + v_l + v_r - 4*v

## Approximation of the non-local kernel by auxiliary components


In [ ]:
c0 = cp.zeros((numSp+1, sizeX, sizeY), dtype=cp.float64)
c0[0] = cp.asarray(_c0)


In [ ]:
c_learn = c0.copy()

def update_learn(c):
    u, v = c[0], c[1:]
    reaction_u = cp.zeros_like(u)
    reaction_v = mu*u - v
    diff_v     = D_vec * laplacian_all(v)
    dudt = reaction_u[None]
    dvdt = (reaction_v + diff_v) / epsilon_learn
    return cp.concatenate([dudt, dvdt])

for t in range(n_learn):
    c_learn += tau_learn * update_learn(c_learn)

U_conv = signal.convolve2d(c_learn[0], K_cp, mode='same', boundary='wrap')

X = c_learn[1:].reshape(numSp, -1).T
y = U_conv.ravel()

Xb = cp.concatenate([X, cp.ones((X.shape[0],1),dtype=X.dtype)], 1)
beta, *_ = cp.linalg.lstsq(Xb, y, rcond=None)

coef_cp = beta[:-1]
icpt_cp = beta[-1]



# Run RD Lenia

In [ ]:
c = c0.copy()

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(7,4))
im0 = ax0.imshow(cp.asnumpy(c[0]), cmap='viridis', origin='lower')
ax0.set_title("step 0")

im1 = ax1.imshow(cp.asnumpy(c[0]), cmap='viridis', origin='lower')
ax1.set_title("step 0 (current)")
plt.show()

snapshots = []

def update_sim(c):
    u, v = c[0], c[1:]
    U_RD = cp.tensordot(v, coef_cp, axes=(0,0)) + icpt_cp
    reaction_u = target_cp(U_RD) - u
    reaction_v = mu*u - v
    diff_v     = D_vec * laplacian_all(v)
    dudt = reaction_u[None]
    dvdt = (reaction_v + diff_v) / epsilon_sim
    return cp.concatenate([dudt, dvdt])

for step in range(n_sim):
    c += tau_sim * update_sim(c)
    if step % 10000 == 0:
        im1.set_data(cp.asnumpy(c[0]))
        ax1.set_title(f"step {step}")
        clear_output(wait=True); display(fig)
